# <div align='center'> 分类任务测试 </div>

In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
backbones = [
    'resnet18',           # 0
    'resnet50',           # 1
    'densenet121',        # 2
    'mobilenet_v2',       # 3
    'squeezenet1_0',      # 4
    'squeezenet1_1',      # 5
    'shufflenet_v2_x0_5', # 6
    'shufflenet_v2_x1_0', # 7
]

batch_sizes = [8, 16, 32, 64, 128, 256]
input_sizes = [28, 32, 64, 128, 256, 512]

## 任务

In [5]:
from k12libs.utils.nb_easy import k12ai_run_project
project = k12ai_run_project(
    debug=True, tb_port=9002,
    framework='k12cv', task='cls',
    network=backbones[-1], dataset='mnist')

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…

## 压测 (P40 跑10个任务)

In [3]:
from k12libs.utils.nb_easy import k12ai_train_execute
keys = k12ai_train_execute(
    framework='k12cv', task='cls',
    network=backbones[-1], dataset='mnist',
    batchsize=32, inputsize=28, epoch_num=2, run_num=10)

## 统计

```json
{
  "version": "0.1.0",
  "ip": "10.255.0.58",
  "type": "error",
  "token": "123456",
  "user": "16601548608",
  "op": "train.start",
  "service_name": "k12cv",
  "service_uuid": "41edf3",
  "timestamp": 1594824029703,
  "datetime": "20200715224029",
  "data": {
    "code": 100003,
    "message": {
      "en": "program normal exit",
      "cn": "\u7a0b\u5e8f\u6b63\u5e38\u7ed3\u675f"
    },
    "expand": {
      "status": "finish",
      "uptime": 27,
      "memstat": {
        "app_cpu_memory_usage_MB": 4271.531,
        "app_gpu_memory_usage_MB": 34.071,
        "sys_cpu_memory_free_MB": 42289.039,
        "sys_gpu_memory_free_MB": [
          22282.0
        ],
        "peak_cpu_self_ru_maxrss": 2367.492,
        "peak_cpu_children_ru_maxrss": 1904.039,
        "peak_gpu_0_memory_cached_MB": 46.0,
        "peak_gpu_0_memory_allocated_MB": 19.522,
        "peak_gpu_0_max_memory_cached_MB": 46.0,
        "peak_gpu_0_max_memory_allocated_MB": 34.071
      },
      "resource": {
        "containers": [
          {
            "id": "f19713b1122e",
            "op": "train.start",
            "user": "16601548608",
            "service_uuid": "41edf3",
            "cpu_percent": 0.13,
            "cpu_memory_total_MB": 10240.0,
            "cpu_memory_usage_MB": 1889.574,
            "cpu_memory_percent": 18.45,
            "gpu_percent": 0.0,
            "gpu_memory_usage_MB": 627,
            "gpu_memory_percent": 0.0
          }
        ]
      },
      "environ": {
        "dataset_name": "mnist",
        "model_name": "shufflenet_v2_x1_0",
        "batch_size": "32",
        "input_size": "[28, 28]"
      }
    }
  }
}
```

In [ ]:
from k12libs.utils.nb_easy import k12ai_train_execute
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_print
import json

max_try = 1800
stats = []
for net in backbones:
    for bs in batch_sizes:
        for ss in input_sizes:
            print(net, bs, ss)
            trycnt = max_try
            keys = k12ai_train_execute(
                framework='k12cv', task='cls',
                network=net, dataset='mnist',
                batchsize=bs, inputsize=ss, epoch_num=2, run_num=1)
            while trycnt > 0:
                data = k12ai_get_data(keys[0], 'error', num=1, rm=True, waitcnt=2)
                if data:
                    try:
                        result = data[0]['value']
                        code = result['data']['code']
                        if code == 100004 or code > 100100: # stop or error
                            print('-'*40, '\n', result)
                            break
                        elif code == 100003: # finish
                            stats.append(result)
                            print('-'*40, '\n', stats[-1])
                            break
                    except Exception as err:
                        print(err, keys[0])
                trycnt += 1
                
with open('/data/mnist_stats.json', 'w') as f:
    json.dump(stats, f)

resnet18 8 28
---------------------------------------- 
 {'version': '0.1.0', 'ip': '10.255.0.58', 'type': 'error', 'token': '3f6fe7', 'user': '15801310416', 'op': 'train.start', 'service_name': 'k12cv', 'service_uuid': '3f6fe7-0', 'timestamp': 1594827552041, 'datetime': '20200715233912', 'data': {'code': 100003, 'message': {'en': 'program normal exit', 'cn': '程序正常结束'}, 'expand': {'status': 'finish', 'uptime': 70, 'memstat': {'app_cpu_memory_usage_MB': 4339.554, 'app_gpu_memory_usage_MB': 223.357, 'sys_cpu_memory_free_MB': 42148.191, 'sys_gpu_memory_free_MB': [22084.0], 'peak_cpu_self_ru_maxrss': 2366.609, 'peak_cpu_children_ru_maxrss': 1972.945, 'peak_gpu_0_memory_reserved_MB': 244.0, 'peak_gpu_0_memory_allocated_MB': 173.43, 'peak_gpu_0_max_memory_reserved_MB': 286.0, 'peak_gpu_0_max_memory_allocated_MB': 223.357}, 'resource': {'containers': [{'id': '159d2f00da7d', 'op': 'train.start', 'user': '15801310416', 'service_uuid': '3f6fe7-0', 'service_starttime': '2020-07-15T15:37:53.639847

---------------------------------------- 
 {'version': '0.1.0', 'ip': '10.255.0.58', 'type': 'error', 'token': '8f3474', 'user': '15801310416', 'op': 'train.start', 'service_name': 'k12cv', 'service_uuid': '8f3474-0', 'timestamp': 1594828224691, 'datetime': '20200715235024', 'data': {'code': 100003, 'message': {'en': 'program normal exit', 'cn': '程序正常结束'}, 'expand': {'status': 'finish', 'uptime': 60, 'memstat': {'app_cpu_memory_usage_MB': 4268.727, 'app_gpu_memory_usage_MB': 1336.24, 'sys_cpu_memory_free_MB': 42102.77, 'sys_gpu_memory_free_MB': [19930.0], 'peak_cpu_self_ru_maxrss': 2366.176, 'peak_cpu_children_ru_maxrss': 1902.551, 'peak_gpu_0_memory_reserved_MB': 2398.0, 'peak_gpu_0_memory_allocated_MB': 173.305, 'peak_gpu_0_max_memory_reserved_MB': 2398.0, 'peak_gpu_0_max_memory_allocated_MB': 1336.24}, 'resource': {'containers': [{'id': 'e01db8a0bc5b', 'op': 'train.start', 'user': '15801310416', 'service_uuid': '8f3474-0', 'service_starttime': '2020-07-15T15:49:17.05246598Z', 'cpu_p

In [ ]:
timestr = '2020-07-15T15:40:46.120656084Z'

